# Beer Recommender System

Building a Crowdsourced Recommender System



In [2]:
import sys
sys.path.insert(0,'chromedriver')
import os 
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# importing packages for NLP
import re
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk import word_tokenize, FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# importing packages to scrape web
from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
stop_words = stopwords.words('english') 

# Web scraping

Task A. Extract about 5-6k reviews.

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Set up Chrome options (chrome_options)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Initialize the WebDriver with Chrome
driver = webdriver.Chrome(options=chrome_options)

url_list = ['https://www.beeradvocate.com/beer/popular/','https://www.beeradvocate.com/beer/worst/',\
            'https://www.beeradvocate.com/beer/top-rated/','https://www.beeradvocate.com/beer/fame/','https://www.beeradvocate.com/beer/top-styles/','https://www.beeradvocate.com/beer/trending/']

correct_links = []
for a in url_list:
    driver.get(a)
    links_list = []
    links = driver.find_elements(By.TAG_NAME,'a')
    for lnk in links:
       links_list.append(lnk.get_attribute('href'))
        
    for i in links_list:
        if "/profile/" in str(i) and i.count('/') == 7:
            correct_links.append(i)
correct_links_nd = list(set(correct_links))

driver.quit()



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome options (chrome_options)
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Initialize the WebDriver with Chrome
driver = webdriver.Chrome(options=chrome_options)  # Use options as a keyword argument

comments = []
product_name = []

# Increase the timeout value to allow for longer page load times
wait = WebDriverWait(driver, 30)

for j in range(len(correct_links_nd)):
    driver.get(str(correct_links_nd[j]))
    
    # Wait for the titleBar element to be present on the page
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'titleBar')))
    
    beer = driver.find_element(By.CLASS_NAME, 'titleBar').text.split('\n')[0]
    review = driver.find_elements(By.CLASS_NAME, 'user-comment')
    for i in review:
        comment = i.find_element(By.ID, 'rating_fullview_content_2').text
        comments.append(comment)
        product_name.append(beer)
    print(j)

driver.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# removes extra data that comes with each comment
def split_comment(s):
    split_result = s.split('\n')
    comment_body = split_result[5:-2]
    return comment_body

#selecting only beer profile links not company/place profiles
def altLinks(l):
    return l[::2]

# converts comments to a string and removes brackets
def list_to_string(x):
    string = str(x)
    return string.rstrip(']').lstrip('[')

# Function to replace \n and "," with blanks('')
def replacenewline(x):
    return(str(x).replace('\n', ' ').replace(",", ""))

# Function to replace "\'" with blanks('')
def replaceapostrophe(z):
    return(str(z).replace("'", "").replace('"', ""))

In [ ]:
#Cleaning the scraped data

df = pd.DataFrame(columns = ["product_name", "user_rating","raw_comments", "product_review"])
df['product_name'] = product_name
df['raw_comments'] = comments
string = 'Rated: '
df['user_rating'] = df['raw_comments'].map(lambda x: x.split('\n')[2].split('/')[0] if(string not in x) else np.nan)
df['product_review'] = df['raw_comments'].map(lambda x: (' '.join(x.split('\n')[4:-1])).lower() if ('Rated: ' not in x) else np.nan)
df.drop(columns = ['raw_comments'], inplace = True)
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
df['product_review'] = df['product_review'].map(lambda x: re.sub(r'[^\w\s]', '', x.replace("\n", " ")))
df.to_csv('Comments_from_top_rated.csv')
df

## Task B. 
Assume that a customer, who will be using this recommender system, has specified 3 attributes
in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes
from the actual data)
https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/
* Aggressive (Boldly assertive aroma and/or taste)
* Balanced: Malt and hops in similar proportions; equal representation of malt sweetness and
hop bitterness in the flavor — especially at the finish
* Complex: Multidimensional; many flavors and sensations on the palate
* Crisp: Highly carbonated; effervescent
* Fruity: Flavors reminiscent of various fruits or Hoppy: Herbal, earthy, spicy, or citric aromas and
flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry
* Robust: Rich and full-bodied

  
Use the above attributes as examples only, for a word frequency analysis of beer reviews is a better way
to find important attributes in the actual data.
Assume that a customer has specified 3 attributes of the product as being important to him or her



In [26]:
df = pd.read_csv("Comments_from_top_rated.csv", index_col=0)
df.head()

,product_name,user_rating,product_review
0,Broo Doo,4.00,bottled 992022 the first thing that comes to m...
1,Broo Doo,4.24,12oz bottle etched on bottle neck is bottled 0...
2,Broo Doo,4.15,6 oz minisnifter draft pour at lures bar gril...
3,Foster's Lager,3.00,i had this beer a couple of times before and a...
4,Foster's Lager,3.82,yum


In [27]:
df_copy=df
df_copy

,product_name,user_rating,product_review
0,Broo Doo,4.00,bottled 992022 the first thing that comes to m...
1,Broo Doo,4.24,12oz bottle etched on bottle neck is bottled 0...
2,Broo Doo,4.15,6 oz minisnifter draft pour at lures bar gril...
3,Foster's Lager,3.00,i had this beer a couple of times before and a...
4,Foster's Lager,3.82,yum
...,...,...,...
6836,IPA,4.00,served up in a pint sparky and sting notes of ...
6837,IPA,3.80,poured from a 12oz can on the pour 3 fingers ...
6838,IPA,4.27,from pilgrims market in cda idaho 12 ounce ca...
6839,IPA,4.25,12oz can date stamp on bottom lip looks to be ...


The code is cleaning and analyzing the product reviews to identify and count the most frequent adjectives used to describe various attributes of the products. The attributes list is used to focus specifically on words that describe product characteristics, helping to understand what customers often mention in their reviews.

In [28]:
reviews = df.product_review.copy()
regex_pattern = "[^a-zA-Z\s]"
df.loc[:,'cleaned_review'] = df.product_review.str.lower().str.replace(pat=regex_pattern,repl='',regex=True) 

In [29]:
# Overall Word Frequency
entire_corpus = df.cleaned_review.str.cat(sep=' ') # Entire corpus in one big string
all_tokens = nltk.word_tokenize(entire_corpus) # Tokenize everything
tokens_no_stop_words = [token for token in all_tokens if token not in stop_words] # Remove stop words from all tokens
word_counts_all = pd.DataFrame(data=nltk.FreqDist(tokens_no_stop_words).most_common(), columns=['word','frequency'])

tagged_tokens = nltk.pos_tag(tokens_no_stop_words) 
adjectives_only = [word for word, tag in tagged_tokens if tag in ['JJ','JJR','JJS']] # Filter for adjectives
word_counts_adjectives = pd.DataFrame(data=nltk.FreqDist(adjectives_only).most_common(), columns=['word','frequency']) 


In [30]:
#List of Attributes identified for this analysis 
attributes = ['light','sweet','clear','dark','golden','strong','black','brown','bitter','smooth','moderate','floral','earthy','creamy','pale','yellow','fresh','bready','malt','mild','bourbon','chocolate','citrus','coffee','mouthfeel','hazy','vanilla','orange','roasted','tropical','fruit','thick','caramel','maple','hops']
word_counts_adjectives[word_counts_adjectives['word'].isin(attributes)].sort_values(by='frequency', ascending=False)[:20]

,word,frequency
1,light,1922
2,sweet,1809
9,dark,1060
10,clear,988
14,black,782
15,golden,740
16,strong,717
18,bitter,645
19,brown,642
21,smooth,604


With the below code, we aim to identify and quantify specific attributes that customers associate with our products based on their reviews, allowing us to gain insights into the key qualities that matter most to our customers and guide product development and marketing strategies accordingly.
The specific attributes we are trying to identify and analyze in the code are "Aggressive," "Balanced," "Complex," "Crisp," "Fruity," "Hoppy," and "Robust," which are characteristics associated with beer products.


In [31]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Load the dataset
df = pd.read_csv("Comments_from_top_rated.csv", index_col=0)

# Define the attributes to search for
attributes = ['Aggressive', 'Balanced', 'Complex', 'Crisp', 'Fruity', 'Hoppy', 'Malts', 'Robust']

# Initialize a dictionary to store attribute counts
attribute_counts = {attr: 0 for attr in attributes}

# Process each review
for review in df['product_review']:
    if pd.notna(review):  # Check if the review is not NaN
        # Convert the review to lowercase and remove non-alphabetical characters
        cleaned_review = ''.join(c.lower() if c.isalpha() or c.isspace() else ' ' for c in review)
        
        # Tokenize the cleaned review
        tokens = nltk.word_tokenize(cleaned_review)
        
        # Check if any attribute is present in the review
        for attr in attributes:
            if attr.lower() in tokens:
                attribute_counts[attr] += 1

# Convert the attribute counts to a DataFrame
attribute_counts_df = pd.DataFrame.from_dict(attribute_counts, orient='index', columns=['Count'])

# Sort the attributes by count in descending order
attribute_counts_df = attribute_counts_df.sort_values(by='Count', ascending=False)

# Display the counts of each attribute
print(attribute_counts_df)


            Count
Malts         503
Crisp         469
Balanced      440
Fruity        353
Complex       299
Hoppy         286
Aggressive     63
Robust         55


## Task C
Perform a similarity analysis using cosine similarity (without word embeddings) with the 3
attributes specified by the customer and the reviews.
The similarity script should accept as input a file with the product attributes, and calculate similarity
scores (between 0 and 1) between these attributes and each review. That is, the output file should have
3 columns – product_name (for each product, the product_name will repeat as many times as there are
reviews of the product), product_review and similarity_score

The below code, calculates similarity scores between three specified product attributes and a collection of product reviews using the cosine similarity measure. It combines the attributes and reviews into a common representation, calculates their vectorized representations using TF-IDF, and then computes the cosine similarity between attribute vectors and review vectors, storing the results in a DataFrame and saving them to a CSV file.
* The attributes used in the code are "light," "sweet," and "dark."


In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset with attributes and reviews
df = pd.read_csv("Comments_from_top_rated.csv")

# Define the 3 attributes specified by the customer
attributes = ['light', 'sweet', 'dark']

# Fill NaN values in the 'product_review' column with empty strings
df['product_review'].fillna('', inplace=True)

# Preprocess attributes and reviews
# Combine attributes and reviews into a single list
texts = attributes + df['product_review'].str.lower().tolist()

# Create TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

# Calculate cosine similarity between attributes and reviews
similarity_scores = cosine_similarity(tfidf_matrix[:len(attributes)], tfidf_matrix[len(attributes):])

# Create an empty list to store the results
results = []

# Iterate over reviews and attributes to store the results
for i, attribute in enumerate(attributes):
    for j, review in enumerate(df['product_review']):
        similarity_score = similarity_scores[i][j]  # Adjusted index to match attributes and reviews
        results.append({
            'product_name': df.iloc[j]['product_name'],
            'product_review': review,
            'similarity_score': similarity_score
        })

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('similarity_scores.csv', index=False)
results_df.sort_values(by=['similarity_score'], ascending = False)


,product_name,product_review,similarity_score
10825,Stickee Monkee,sweet too sweet,0.844851
6256,Budweiser Select 55,its light,0.703622
14730,Tucher Nürnberger Rotbier,pours a dark hazy brown color with a generous ...,0.482864
6646,Heineken Premium Light Lager,a decent light beer,0.477981
5498,Old Milwaukee Light,i usually drink labatt light blue royal or mil...,0.473789
...,...,...,...
7980,Saison Dupont,pours mid amber gold mildly cloudy with some f...,0.000000
7981,Saison Dupont,this is a unique and funky adventure hay barna...,0.000000
7982,Saison Dupont,71523 on tap at apothecary alehouse in morgant...,0.000000
7983,Saison Dupont,16oz undated can pours clear gold with three ...,0.000000


## Task D
For every review, perform a sentiment analysis (using VADER or any LLM). 


### Sentiment analysis using Vader

In [12]:
pip install vaderSentiment


     -------------------------------------- 126.0/126.0 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd

# Load the CSV files into DataFrames
similarity_df = pd.read_csv("similarity_scores.csv")
reviews_df = pd.read_csv("comments_from_top_rated.csv")

# Merge the two DataFrames based on 'product_name'
merged_df = pd.merge(reviews_df, similarity_df, on='product_name', how='left')

# Drop one of the duplicate columns
merged_df.drop(columns='product_review_x', inplace=True)

# Rename the remaining column to 'product_review'
merged_df.rename(columns={'product_review_y': 'product_review'}, inplace=True)

# Filter the desired columns
desired_columns = ['product_name', 'product_review', 'similarity_score']

# Display the merged DataFrame with the specified columns
print(merged_df[desired_columns])


       product_name                                     product_review  \
0          Broo Doo  bottled 992022 the first thing that comes to m...   
1          Broo Doo  12oz bottle etched on bottle neck is bottled 0...   
2          Broo Doo  6 oz minisnifter draft pour at lures bar  gril...   
3          Broo Doo  bottled 992022 the first thing that comes to m...   
4          Broo Doo  12oz bottle etched on bottle neck is bottled 0...   
...             ...                                                ...   
192586          IPA  served up in a pint sparky and sting notes of ...   
192587          IPA  poured from a 12oz can  on the pour 3 fingers ...   
192588          IPA  from pilgrims market in cda idaho  12 ounce ca...   
192589          IPA  12oz can date stamp on bottom lip looks to be ...   
192590          IPA  from a stovepipe as mother nature intended pou...   

        similarity_score  
0                    0.0  
1                    0.0  
2                    0.0  
3  

In [34]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
# Initialize an empty list to store sentiment scores
sentiment_scores = []

# Iterate over the reviews in the DataFrame
for review in merged_df['product_review']:
    if pd.notna(review):
        # Perform sentiment analysis using VADER
        sentiment_score = analyzer.polarity_scores(review)
        sentiment_scores.append(sentiment_score)
    else:
        # Handle NaN values with None or other suitable approach
        sentiment_scores.append(None)

# Add the sentiment scores as a new column in the DataFrame
merged_df['sentiment_scores'] = sentiment_scores
merged_df['overall_sentiment_score'] = merged_df['sentiment_scores'].apply(lambda x: x['compound'] if x is not None else None)

# Display the merged DataFrame with sentiment scores
print(merged_df[['product_name', 'product_review', 'similarity_score', 'overall_sentiment_score']])


       product_name                                     product_review  \
0          Broo Doo  bottled 992022 the first thing that comes to m...   
1          Broo Doo  12oz bottle etched on bottle neck is bottled 0...   
2          Broo Doo  6 oz minisnifter draft pour at lures bar  gril...   
3          Broo Doo  bottled 992022 the first thing that comes to m...   
4          Broo Doo  12oz bottle etched on bottle neck is bottled 0...   
...             ...                                                ...   
192586          IPA  served up in a pint sparky and sting notes of ...   
192587          IPA  poured from a 12oz can  on the pour 3 fingers ...   
192588          IPA  from pilgrims market in cda idaho  12 ounce ca...   
192589          IPA  12oz can date stamp on bottom lip looks to be ...   
192590          IPA  from a stovepipe as mother nature intended pou...   

        similarity_score  overall_sentiment_score  
0                    0.0                   0.5867  
1      

In [35]:
# Save the DataFrame as a tab-separated values (TSV) file
merged_df.to_csv('output.tsv', sep='\t', index=False)
merged_df[['product_name', 'product_review', 'similarity_score', 'overall_sentiment_score']]

,product_name,product_review,similarity_score,overall_sentiment_score
0,Broo Doo,bottled 992022 the first thing that comes to m...,0.0,0.5867
1,Broo Doo,12oz bottle etched on bottle neck is bottled 0...,0.0,0.9628
2,Broo Doo,6 oz minisnifter draft pour at lures bar gril...,0.0,0.7506
3,Broo Doo,bottled 992022 the first thing that comes to m...,0.0,0.5867
4,Broo Doo,12oz bottle etched on bottle neck is bottled 0...,0.0,0.9628
...,...,...,...,...
192586,IPA,served up in a pint sparky and sting notes of ...,0.0,0.2484
192587,IPA,poured from a 12oz can on the pour 3 fingers ...,0.0,0.9856
192588,IPA,from pilgrims market in cda idaho 12 ounce ca...,0.0,0.8963
192589,IPA,12oz can date stamp on bottom lip looks to be ...,0.0,0.8126


## Task E

Create an evaluation score for each beer that uses both similarity and sentiment scores.
Now recommend 3 products to the customer

In [36]:
# Calculate the evaluation score
weight_similarity = 0.6
weight_sentiment = 0.4
merged_df['evaluation_score'] = (merged_df['similarity_score'] * weight_similarity) + (merged_df['overall_sentiment_score'] * weight_sentiment)

# Group by product_name and calculate the mean of user_rating, similarity, sentiment, and evaluation_score
beer_scores = merged_df.groupby('product_name').agg({
    'product_review': 'first',
    'similarity_score': 'mean',
    'overall_sentiment_score': 'mean',
    'evaluation_score': 'mean'
}).reset_index()

# Sort by evaluation_score in descending order
beer_scores = beer_scores.sort_values(by='evaluation_score', ascending=False)

# Recommend the top 3 products
top_recommendations = beer_scores.head(3)

# Save the recommendations to a TSV file
top_recommendations.to_csv('recommendations.tsv', sep='\t', index=False)

# Display the top recommendations
(top_recommendations[['product_name', 'product_review', 'similarity_score', 'overall_sentiment_score','evaluation_score']])


,product_name,product_review,similarity_score,overall_sentiment_score,evaluation_score
906,Wide Awake It's Morning,2017 vintage moving right along from the incr...,0.076245,0.977600,0.436787
459,It Was All A Dream,pours very dark with minimal head has occasion...,0.110762,0.923100,0.435697
865,Trois Pistoles,what a good looking pour on this one drank fro...,0.078590,0.950357,0.427297


Here are the top three product recommendations based on a combined evaluation score that considers both similarity and sentiment scores:

* Wide Awake It's Morning

Similarity Score: The similarity score for this product is 0.076, indicating some similarity to other products.
Overall Sentiment Score: The overall sentiment score is impressively high at 0.978, suggesting very positive sentiment.
Evaluation Score: The combined evaluation score, which weighs both similarity and sentiment, is 0.437. This product stands out as a strong recommendation due to its positive sentiment.

* It Was All A Dream

Similarity Score: The similarity score is 0.111, indicating moderate similarity to other products.
Overall Sentiment Score: The overall sentiment score is 0.923, indicating generally positive sentiment.
Evaluation Score: The combined evaluation score for this product is 0.436, making it another solid recommendation based on its combination of similarity and sentiment.

* Trois Pistoles

Similarity Score: The similarity score is 0.075, suggesting some similarity to other products.
Overall Sentiment Score: The overall sentiment score is 0.939, indicating highly positive sentiment.
Evaluation Score: The combined evaluation score for this product is 0.420. It's a strong recommendation, especially for those who prioritize positive sentiment in their choices.


These recommendations are generated by considering both how similar these products are to others and the overall positive sentiment expressed in their reviews. Customers looking for products with these characteristics may find these recommendations appealing.

In [37]:
beer_reco = top_recommendations[:3]['product_name'].to_list()
beer_reco

["Wide Awake It's Morning", 'It Was All A Dream', 'Trois Pistoles']

# Recommender System using word embeddings

In [18]:
!pip install spacy

     --------------------------------------- 12.0/12.0 MB 36.3 MB/s eta 0:00:00
     ---------------------------------------- 122.2/122.2 kB ? eta 0:00:00
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 31.2 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB ? eta 0:00:00
     ------------------------------------- 481.9/481.9 kB 31.4 MB/s eta 0:00:00
     ---------------------------------------- 395.3/395.3 kB ? eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 31.9 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 38.7 MB/s eta 0:00:00


In [39]:
# Downloading the small model containing tensors.
!python -m spacy download en_core_web_sm

# Downloading over 1 million word vectors.
!python -m spacy download en_core_web_lg

     --------------------------------------- 12.8/12.8 MB 31.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     -------------------------------------- 587.7/587.7 MB 1.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [40]:
pip install typing_extensions --upgrade


  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.10.0 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [41]:
import pandas as pd
import spacy

# Sample list of attributes
required_attributes = ['light', 'sweet', 'dark']

# Function to calculate word embedding similarity
def similarity_word_embeddings(review):
    review_nlp = nlp(review)
    scores = [review_nlp.similarity(attr_nlp) for attr_nlp in attributes_nlp]
    average_score = sum(scores) / len(scores)
    return average_score

# Load spaCy's pre-trained word embedding model
nlp = spacy.load('en_core_web_lg')

# Process the attributes
attributes_nlp = [nlp(attr) for attr in required_attributes]

# Assuming you have a DataFrame called df with a 'product_review' column
df['word_embedding_similarity'] = df['product_review'].apply(similarity_word_embeddings)

# Display the DataFrame with the average word embedding similarity scores
df[['product_name', 'product_review', 'word_embedding_similarity']]


C:\Users\dhruv\AppData\Local\Temp\ipykernel_25000\2416810966.py:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  scores = [review_nlp.similarity(attr_nlp) for attr_nlp in attributes_nlp]


,product_name,product_review,word_embedding_similarity
0,Broo Doo,bottled 992022 the first thing that comes to m...,0.402808
1,Broo Doo,12oz bottle etched on bottle neck is bottled 0...,0.458553
2,Broo Doo,6 oz minisnifter draft pour at lures bar gril...,0.384942
3,Foster's Lager,i had this beer a couple of times before and a...,0.392922
4,Foster's Lager,yum,0.053797
...,...,...,...
6836,IPA,served up in a pint sparky and sting notes of ...,0.424599
6837,IPA,poured from a 12oz can on the pour 3 fingers ...,0.438940
6838,IPA,from pilgrims market in cda idaho 12 ounce ca...,0.452703
6839,IPA,12oz can date stamp on bottom lip looks to be ...,0.469882


In [42]:
# Merge 'merged_df' and 'beer_scores' based on 'product_name' with suffixes
merged_df = pd.merge(merged_df, beer_scores[['product_name', 'similarity_score']], on='product_name', how='left', suffixes=('_merged', '_beer_scores'))

# Assuming you have 'user_rating' in 'merged_df'

# Select the desired columns
selected_columns = ['product_name', 'user_rating', 'product_review', 'similarity_score_merged']

# Create a DataFrame with the selected columns
final_df = merged_df[selected_columns]

# Optionally, you can sort the DataFrame by 'similarity_score_merged' in descending order
final_df = final_df.sort_values(by='similarity_score_merged', ascending=False)

# Display the top rows of the final DataFrame
final_df.head(5)


,product_name,user_rating,product_review,similarity_score_merged
108611,Stickee Monkee,4.50,sweet too sweet,0.844851
108575,Stickee Monkee,4.67,sweet too sweet,0.844851
108935,Stickee Monkee,4.56,sweet too sweet,0.844851
108683,Stickee Monkee,4.16,sweet too sweet,0.844851
108791,Stickee Monkee,4.22,sweet too sweet,0.844851


In [43]:
# Merge 'beer_scores' and 'df' based on 'product_name'
beer_scores = pd.merge(beer_scores, df[['product_name', 'word_embedding_similarity']], on='product_name', how='left')

# Display the updated 'beer_scores' DataFrame
#beer_scores.head(5)
beer_scores[beer_scores['product_name']=='Blended With Lumber #1']

,product_name,product_review,similarity_score,overall_sentiment_score,evaluation_score,word_embedding_similarity
1529,Blended With Lumber #1,pours a dark brown color with a minimal head t...,0.067752,0.70815,0.323911,0.525481
1530,Blended With Lumber #1,pours a dark brown color with a minimal head t...,0.067752,0.70815,0.323911,0.521880


In [44]:
# Assuming 'beer_scores' is your DataFrame
df_we_similarity = beer_scores[['product_name', 'similarity_score', 'overall_sentiment_score', 'word_embedding_similarity']].copy()
top_beer_reco_word_embedding = df_we_similarity.groupby(by='product_name').mean().sort_values(by='word_embedding_similarity', ascending=False)[:3].index.to_list()
df_we_similarity.groupby(by='product_name').mean().sort_values(by='word_embedding_similarity', ascending=False)[:3]


,similarity_score,overall_sentiment_score,word_embedding_similarity
product_name,,,
It Was All A Dream,0.110762,0.92310,0.540504
Saison Du Fermier,0.023411,0.42210,0.539382
Blended With Lumber #1,0.067752,0.70815,0.523680


In [45]:
top_beer_reco_word_embedding

['It Was All A Dream', 'Saison Du Fermier', 'Blended With Lumber #1']

It appears that the Word Embeddings method performed slightly better in terms of word_embedding_similarity, as it provided higher values for the top recommendations. This suggests that word embeddings captured more nuanced relationships between products based on the meaning of words in their reviews.

1. It Was All A Dream:

* Similarity Score: 0.111
This similarity score of 0.111 indicates that "It Was All A Dream" is moderately similar to other products in the dataset. In terms of characteristics or features, it shares some commonalities with other products.
* Overall Sentiment Score: 0.923
The overall sentiment score is 0.923, suggesting that the reviews for "It Was All A Dream" are generally positive. Customers who have reviewed this product have expressed positive sentiments in their feedback.
* Word Embedding Similarity: 0.540
The word embedding similarity score is 0.540. This score is based on the semantic understanding of the product's reviews and how closely they align with other products in terms of the meaning of words used. It indicates that the product's reviews have a relatively high degree of similarity to other products based on the meaning of words.

2. Saison Du Fermier:

* Similarity Score: 0.023
The similarity score for "Saison Du Fermier" is 0.023, suggesting that it has lower similarity to other products in the dataset. It might have unique characteristics that set it apart from most other products.
* Overall Sentiment Score: 0.422
The overall sentiment score is 0.422, indicating that the reviews for this product have a relatively lower overall sentiment. The sentiment expressed in these reviews tends to be less positive compared to other products.
* Word Embedding Similarity: 0.539
With a word embedding similarity score of 0.539, "Saison Du Fermier" has a relatively high degree of similarity with other products based on the meaning of words used in reviews. This suggests that the product's reviews share semantic similarities with other products.

3. Blended With Lumber #1:

* Similarity Score: 0.068
The similarity score of 0.068 suggests that "Blended With Lumber #1" has moderate similarity to other products in the dataset. It shares some commonalities with other products in terms of characteristics or features.
* Overall Sentiment Score: 0.708
The overall sentiment score is 0.708, indicating that the reviews for this product tend to be positive. Customers who have reviewed this product have expressed relatively positive sentiments.
* Word Embedding Similarity: 0.524
The word embedding similarity score is 0.524, indicating that the reviews for "Blended With Lumber #1" have a moderate degree of similarity with other products based on the meaning of words. The meaning of words used in its reviews aligns moderately with other products.


In summary, these scores provide insights into how similar these products are to others, the overall sentiment expressed in their reviews, and the semantic similarity of their reviews based on the meaning of words.

### Task F
How would your recommendation change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute.
E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? Explain. 

In [46]:
merged_df_cos=merged_df

merged_df_cos['aggregated_reviews'] = merged_df_cos.groupby('product_name')['product_review'].transform(lambda x: '\k'.join(map(str, x)))

merged_new=merged_df_cos[['product_name','aggregated_reviews']].drop_duplicates()


aggregated_reviews_df_cosine = pd.merge(merged_new, top_recommendations, left_on='product_name', right_on='product_name', how='inner')


In [47]:
aggregated_reviews_df_cosine

,product_name,aggregated_reviews,product_review,similarity_score,overall_sentiment_score,evaluation_score
0,Wide Awake It's Morning,2017 vintage moving right along from the incr...,2017 vintage moving right along from the incr...,0.076245,0.977600,0.436787
1,Trois Pistoles,what a good looking pour on this one drank fro...,what a good looking pour on this one drank fro...,0.078590,0.950357,0.427297
2,It Was All A Dream,pours very dark with minimal head has occasion...,pours very dark with minimal head has occasion...,0.110762,0.923100,0.435697


In [48]:
#aggregated_reviews_df_cosine.drop(['similarity_score','overall_sentiment_score','evaluation_score'],axis=1,inplace=True)
aggregated_reviews_df_cosine


,product_name,aggregated_reviews,product_review,similarity_score,overall_sentiment_score,evaluation_score
0,Wide Awake It's Morning,2017 vintage moving right along from the incr...,2017 vintage moving right along from the incr...,0.076245,0.977600,0.436787
1,Trois Pistoles,what a good looking pour on this one drank fro...,what a good looking pour on this one drank fro...,0.078590,0.950357,0.427297
2,It Was All A Dream,pours very dark with minimal head has occasion...,pours very dark with minimal head has occasion...,0.110762,0.923100,0.435697


In [49]:
df_we_similarity1=df_we_similarity.groupby(by='product_name').mean().sort_values(by='word_embedding_similarity', ascending=False)[:3]

merged_df_embed=beer_scores
merged_df_embed['aggregated_reviews'] = merged_df_embed.groupby('product_name')['product_review'].transform(lambda x: '\k'.join(map(str, x)))

merged_new_embed=merged_df_embed[['product_name','aggregated_reviews']].drop_duplicates()


aggregated_reviews_df_embed = pd.merge(merged_new_embed,df_we_similarity1 , left_on='product_name', right_on='product_name', how='inner')
aggregated_reviews_df_embed.drop(['word_embedding_similarity','overall_sentiment_score','similarity_score'],axis=1,inplace=True)
aggregated_reviews_df_embed.to_csv('comments_embdeeing.csv')

In [50]:
import pandas as pd
import re  # Importing the regex module to tokenize the reviews into words

given_attributes = ['sweet', 'light', 'dark']

def calculate_individual_attribute_mention(aggregated_reviews, attribute):
    aggregated_reviews = str(aggregated_reviews)
    review_list = aggregated_reviews.split('\k')
    count = 0
    for review in review_list:
        words = re.findall(r'\b\w+\b', review.lower())  # Tokenizing the review into words and making the match case-insensitive
        if attribute in words:  # Checking if attribute is in the list of words
            count += 1
    percentage = (count / len(review_list)) * 100 if review_list else 0
    return percentage

def create_attribute_mention_df(df):
    rows_list = []
    for index, row in df.iterrows():
        for attribute in given_attributes:
            percentage = calculate_individual_attribute_mention(row['aggregated_reviews'], attribute)
            dict1 = {
                'product_name': row['product_name'],
                'attribute': attribute,
                'attribute_mention_%': percentage
            }
            rows_list.append(dict1)

    return pd.DataFrame(rows_list)

def create_pivot_table(attribute_mention_df):
    return attribute_mention_df.pivot_table(index='product_name', columns='attribute', values='attribute_mention_%', fill_value=0)

# assuming aggregated_reviews_df_cosine and aggregated_reviews_df_embed are your DataFrames with aggregated_reviews
attribute_mention_df1 = create_attribute_mention_df(aggregated_reviews_df_cosine)
pivot_table1 = create_pivot_table(attribute_mention_df1)
print("Cosine Similarity:")
print(pivot_table1)

attribute_mention_df2 = create_attribute_mention_df(aggregated_reviews_df_embed)
pivot_table2 = create_pivot_table(attribute_mention_df2)
print("Word Embedding:")
print(pivot_table2)


Cosine Similarity:
attribute                dark       light       sweet
product_name                                         
It Was All A Dream        100  100.000000  100.000000
Trois Pistoles            100   28.571429   57.142857
Wide Awake It's Morning   100  100.000000  100.000000
Word Embedding:
attribute               dark  light  sweet
product_name                              
Blended With Lumber #1   100      0    100
It Was All A Dream       100    100    100
Saison Du Fermier          0    100      0


## to do

### Task G
How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question

In [51]:
top_3_rated_beers = df_copy.groupby('product_name')['user_rating'].mean().sort_values(ascending=False).reset_index()['product_name'][:3]
df_copy.groupby('product_name')['user_rating'].mean().sort_values(ascending=False)[:3]

product_name
Kentucky Brunch Brand Stout                            4.934000
Kentucky Brunch Brand Stout - Double Barrel Reserve    4.916667
Speedway Stout - Vietnamese Coffee                     4.900000
Name: user_rating, dtype: float64

In [52]:

merged_df_2 = df_copy.merge(df_we_similarity, on='product_name', how='inner')

In [53]:

result = merged_df_2[merged_df_2['product_name'].isin(top_3_rated_beers)].groupby('product_name')[['similarity_score', 'overall_sentiment_score', 'word_embedding_similarity']].mean()

result


,similarity_score,overall_sentiment_score,word_embedding_similarity
product_name,,,
Kentucky Brunch Brand Stout,0.008649,0.681980,0.402671
Kentucky Brunch Brand Stout - Double Barrel Reserve,0.000000,0.806933,0.357979
Speedway Stout - Vietnamese Coffee,0.000000,0.000000,0.061819


If we consider the specific products with the highest ratings in the dataset based on the given scores:

* Kentucky Brunch Brand Stout:

Similarity Score (Cosine Similarity): 0.008649

Overall Sentiment Score: 0.681980

Word Embedding Similarity: 0.402671

* Kentucky Brunch Brand Stout - Double Barrel Reserve:

Similarity Score (Cosine Similarity): 0.000000

Overall Sentiment Score: 0.806933

Word Embedding Similarity: 0.357979

* Speedway Stout - Vietnamese Coffee:

Similarity Score (Cosine Similarity): 0.000000

Overall Sentiment Score: 0.000000

Word Embedding Similarity: 0.061819

Let's analyze how these products would meet the requirements of the user looking for recommendations:

Kentucky Brunch Brand Stout has a moderately high overall sentiment score (0.681980) and a relatively good word embedding similarity (0.402671) but a low similarity score (0.008649). This suggests that while it is generally well-received by users and has some relevance to the specified attributes, it may not strongly align with the attribute requirements.

Kentucky Brunch Brand Stout - Double Barrel Reserve has a high overall sentiment score (0.806933) but a very low similarity score (0.000000) and a moderate word embedding similarity (0.357979). While users highly appreciate it, it may not be closely related to the desired attributes, especially regarding similarity.

Speedway Stout - Vietnamese Coffee has low scores in both similarity (0.000000) and overall sentiment (0.000000) but a slightly higher word embedding similarity (0.061819). This product may not strongly align with user expectations for the specified attributes.

In summary, if we were to recommend products solely based on their high overall ratings and without considering attribute-specific metrics, the recommendations might not fully meet the user's requirements for attributes like "light," "sweet," or "dark." While these products enjoy high overall sentiment and popularity, they may not be the best matches in terms of attribute relevance, as indicated by their low similarity scores and word embedding similarities.

Therefore, it's advisable to consider both overall ratings and attribute-specific metrics to provide recommendations that align more closely with the user's desired attributes and preferences.

### Task H
Using the top four attributes of beer (from word frequency analysis), calculate the lifts between these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among the remaining 9) using the lift values. Explain your method

In [54]:
top_attr = ['light','sweet','dark','clear']


In [55]:
top_10_beer = df_copy.groupby('product_name')['product_name'].count().sort_values(ascending=False)[:10]
top_10_beer = top_10_beer.index.to_list()
top_10_beer

['Oktoberfest',
 'IPA',
 'Porter',
 'Oktoberfest Märzen',
 'Tripel',
 'Earthquake High Gravity Lager (10%)',
 'Péché Mortel',
 'Pale Ale',
 'Aecht Schlenkerla Weichsel',
 'Boatswain Double IPA (Twin Screw Steamer)']

# to do

In [60]:
# 1. Compute the Frequency for each attribute and each of the top 10 beers.

# Create binary columns for each attribute (1 if the review contains the attribute, 0 otherwise)
for attr in top_attr:
    df_copy[attr] = df_copy['product_review'].str.contains(attr, case=False, na=False).astype(int)

# Filter the dataframe to only consider the top 10 beers
filtered_df = df_copy[df_copy['product_name'].isin(top_10_beer)]

# 2. Calculate the Lift for each pair of attribute and beer.

# Compute probabilities
prob_attr = filtered_df[top_attr].mean()
prob_beers = filtered_df['product_name'].value_counts(normalize=True)

lift_values = pd.DataFrame(index=top_10_beer, columns=top_attr)

for beer in top_10_beer:
    for attr in top_attr:
        # Joint probability that a review of this beer mentions this attribute
        joint_prob = len(filtered_df[(filtered_df['product_name'] == beer) & (filtered_df[attr] == 1)]) / len(filtered_df)
        # Lift calculation
        lift_values.at[beer, attr] = joint_prob / (prob_attr[attr] * prob_beers[beer])

# 3. Find the Most Similar Beer for a chosen beer based on the lift values.

# Choose the first beer from the top 10 list
chosen_beer = top_10_beer[0]

# Calculate average lift values for the remaining 9 beers (excluding the chosen beer)
average_lifts = lift_values.drop(chosen_beer).mean(axis=1)

# Get the beer with the highest average lift value
most_similar_beer = average_lifts.idxmax()

lift_values


,light,sweet,dark,clear
Oktoberfest,1.017321,1.417897,1.101597,1.562264
IPA,0.931081,0.664579,0.397174,0.938776
Porter,1.001131,1.209286,2.68435,0.634483
Oktoberfest Märzen,1.30975,1.845753,0.512146,2.663158
Tripel,0.982312,0.791037,0.0,0.0
Earthquake High Gravity Lager (10%),0.921676,1.252475,0.270299,1.533333
Péché Mortel,1.097878,0.8841,2.575792,0.0
Pale Ale,0.731919,0.2947,0.286199,0.541176
Aecht Schlenkerla Weichsel,1.555328,0.626238,1.520433,0.575
Boatswain Double IPA (Twin Screw Steamer),0.518443,0.469678,0.608173,0.575


In [62]:
chosen_beer, most_similar_beer

('Oktoberfest', 'Oktoberfest Märzen')

The beer "Oktoberfest" has a lift value greater than 1 with the attributes light, sweet, and clear.   

This indicates that the reviews of "Oktoberfest" are more likely to mention these attributes than we would expect by chance.  

The beer "IPA" has a lift value less than 1 for all attributes, indicating that these attributes are mentioned less frequently in reviews of "IPA" than expected if the mentions were random.  

The beer "Porter" has a notably high lift value with the attribute dark, indicating that reviews of "Porter" are very likely to mention it being "dark".  

The beer "Oktoberfest Märzen" has a very high lift value with the attribute clear, suggesting that reviews of this beer frequently mention it being "clear".  

The most similar beer to "Oktoberfest" based on the lift values is "Oktoberfest Märzen".

For each beer, the code calculates the average lift value across all attributes. This average lift value is a representation of how frequently the attributes are mentioned in conjunction with this beer compared to what we would expect if the attributes were mentioned randomly.  

Choose the Beer with the Highest Average Lift Value:  
The beer with the highest average lift value is the one that, on average, has the strongest association with the given attributes (in this case, 'light', 'sweet', 'dark', and 'clear').

In our case:  

We chose "Oktoberfest" as the reference beer.  
Among the remaining 9 beers, "Oktoberfest Märzen" had the highest average lift value across the given attributes.  

So, based on the lift values, "Oktoberfest Märzen" is the most similar beer to "Oktoberfest" among the remaining 9 beers.  
 




